In [128]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import nearest_points
from shapely import affinity
from datetime import datetime

## Set up variables

In [219]:
# url for Chicago's street center lines layer, from their data portal
cdot_cenerlines_shapefile_path = 'https://data.cityofchicago.org/api/geospatial/6imu-meau?method=export&format=Original'

# path to csv file containing columns named (at minimum)"On", "From", "To", and "ID"
on_from_to_list_path = 'on_from_to_list.csv'

# path to folder the processed data should be saved in
save_folder_path = r'processed_files/'

# timestamp for file naming
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

# file paths for final processed data to be exported
file_path_mapped_gpkg = f"{save_folder_path}{current_time}_on_from_to_mapped.gpkg"
file_path_mapped_csv = f"{save_folder_path}{current_time}_on_from_to_mapped.csv"
file_path_unmapped_gpkg = f"{save_folder_path}{current_time}_on_from_to_unmapped.gpkg"
file_path_unmapped_csv = f"{save_folder_path}{current_time}_on_from_to_unmapped.csv"

## Download and prepare city street centerlines shapefile

In [220]:
# Download CDOT streets layer
cdot_centerlines_gdf = gpd.read_file(cdot_cenerlines_shapefile_path)

In [221]:
# Combine street names and types into a single column
# to match the expected format of the on/from/to street lists
gdf = cdot_centerlines_gdf.copy() 
    
gdf['On_Street'] = gdf['STREET_NAM'] + ' ' + gdf['STREET_TYP']

gdf.head(5)

,OBJECTID,FNODE_ID,TNODE_ID,TRANS_ID,PRE_DIR,STREET_NAM,STREET_TYP,SUF_DIR,STREETNAME,L_F_ADD,...,FLAG_STRIN,EWNS_DIR,EWNS_COORD,CREATE_USE,CREATE_TIM,UPDATE_USE,UPDATE_TIM,SHAPE_LEN,geometry,On_Street
0,510,10809,16581,127104,S,YALE,AVE,NaN,1782,0,...,NaN,W,232,EXISTING,1999-01-01,EXISTING,1999-01-01,220.566012,"LINESTRING (1175570.097 1863498.080, 1175577.8...",YALE AVE
1,511,6501,34082,128895,S,COTTAGE GROVE,AVE,NaN,1236,7301,...,NaN,NaN,0,EXISTING,1999-01-01,EXISTING,1999-01-01,664.774607,"LINESTRING (1182822.668 1856787.427, 1182824.9...",COTTAGE GROVE AVE
2,512,15338,22358,142645,S,CAMPBELL,AVE,NaN,1177,10801,...,NaN,W,2500,EXISTING,1999-01-01,EXISTING,1999-01-01,665.378453,"LINESTRING (1161631.239 1832936.206, 1161634.6...",CAMPBELL AVE
3,513,15799,28881,148189,S,SANGAMON,ST,NaN,1696,0,...,NaN,W,932,EXISTING,1999-01-01,EXISTING,1999-01-01,152.564966,"LINESTRING (1172013.812 1831615.472, 1171905.1...",SANGAMON ST
4,514,36407,36534,139728,W,118TH,ST,NaN,1823,1933,...,NaN,S,11800,EXISTING,1999-01-01,EXISTING,1999-01-01,332.691382,"LINESTRING (1165307.502 1826592.692, 1165260.9...",118TH ST


## Map a single corridor based on on, from, and to street names

In [222]:
def scale_linestring(line, scale_length):
    """
    Scale a linestring to a specified length from its midpoint.
    """

    # Calculate the scaling factor
    current_length = line.length
    scaling_factor = scale_length / current_length

    # Scale the line
    midpoint = line.interpolate(0.5, normalized=True)
    scaled_line = affinity.scale(line, xfact=scaling_factor, yfact=scaling_factor, origin=midpoint)

    return scaled_line

In [223]:
def find_nearest_segments(street1_gdf, street2_gdf):
    """
    Find the nearest segment from each street based on their proximity to the other street.
    """
    # Find the segment in street1 that is closest to any point on street2
    min_distance_1 = street1_gdf.distance(street2_gdf.geometry.unary_union).min()
    nearest_segment_1 = street1_gdf[street1_gdf.distance(street2_gdf.geometry.unary_union) == min_distance_1].geometry.iloc[0]

    # Find the segment in street2 that is closest to any point on street1
    min_distance_2 = street2_gdf.distance(street1_gdf.geometry.unary_union).min()
    nearest_segment_2 = street2_gdf[street2_gdf.distance(street1_gdf.geometry.unary_union) == min_distance_2].geometry.iloc[0]
    
    return nearest_segment_1, nearest_segment_2

In [224]:
def extend_segments_to_intersection(segment1, segment2, scale_length=10560):  # 10560 feet = 2 miles
    """
    Extend the two given segments to create an intersection line.
    """
    # Create extended lines from the segments
    extended_line_1 = scale_linestring(segment1, scale_length)
    extended_line_2 = scale_linestring(segment2, scale_length)
    
    # Check intersection of the extended lines
    intersection = extended_line_1.intersection(extended_line_2)
    
    # If they intersect, return the intersection
    if not intersection.is_empty:
        return intersection
    else:
        # If they don't, return both extended lines for visualization
        return extended_line_1, extended_line_2

In [225]:
def get_intersection_point(street1_gdf, street2_gdf, scale_length=2640):  # 2640 ft = 1/2 mile
    """
    Return the intersection point of two streets. If they don't intersect, find the closest features
    and create a virtual intersection by extending the features to the specified scale_length.
    """
    intersection = street1_gdf.geometry.unary_union.intersection(street2_gdf.geometry.unary_union)
    
    # If intersection exists and is a point, return it
    if not intersection.is_empty:
        if intersection.geom_type == "Point":
            return intersection
        # if multiple intersections exist, return only the first one.
        elif intersection.geom_type == "MultiPoint":
            points = [p for p in intersection.geoms]
            return points[0]
    
    # If no intersection, find the closest points and create a virtual intersection
    nearest_segment_1, nearest_segment_2 = find_nearest_segments(street1_gdf, street2_gdf)
    
    virtual_intersection = extend_segments_to_intersection(nearest_segment_1, nearest_segment_2)
    
    return virtual_intersection


In [226]:
def filter_segments_between_points(on_street_gdf, from_intersection, to_intersection):
    """
    Filter the on_street segments based on the orientation of the line formed by the intersections.
    """
    # Determine the orientation of the intersection line
    delta_x = abs(to_intersection.x - from_intersection.x)
    delta_y = abs(to_intersection.y - from_intersection.y)
    
    filtered_segments = []
    
    # If intersection_line is oriented more in the x direction
    if delta_x > delta_y:
        min_x, max_x = sorted([from_intersection.x, to_intersection.x])
        for index, row in on_street_gdf.iterrows():
            midpoint_x = row['geometry'].centroid.x
            if min_x <= midpoint_x <= max_x:
                filtered_segments.append(row['geometry'])
    # If intersection_line is oriented more in the y direction
    else:
        min_y, max_y = sorted([from_intersection.y, to_intersection.y])
        for index, row in on_street_gdf.iterrows():
            midpoint_y = row['geometry'].centroid.y
            if min_y <= midpoint_y <= max_y:
                filtered_segments.append(row['geometry'])
                
    # Convert the list of filtered segments to a GeoDataFrame
    filtered_gdf = gpd.GeoDataFrame(geometry=filtered_segments, crs=on_street_gdf.crs)
    
    return filtered_gdf

In [227]:
    
def extract_street_segments(gdf, on_street, from_street, to_street):
    '''
    Extract the segment of on_street that is between its intersection with from_street and to_street.
    
    on_street, from_street, and to_street are strings representing cleaned official street names found
    in the gdf.  (For example, "Madison St")
    
    gdf is the CDOT Street Centerline shapefile downloaded and converted to a 
    GeoDataFrame.
    '''

    # This version is for the CDOT map base layer, downloaded as a shapefile.
    # It has separate street_nam and street_typ fields.

    # Filter the GeoDataFrame for the given streets, ignoring case 
    on_street_gdf = gdf[gdf['On_Street'].str.lower() == on_street.lower()]
    
    from_street_gdf = gdf[gdf['On_Street'].str.lower() == from_street.lower()]
    
    to_street_gdf = gdf[gdf['On_Street'].str.lower() == to_street.lower()]
    
    # Get the intersection points
    on_from_point = get_intersection_point(on_street_gdf, from_street_gdf)
    on_to_point = get_intersection_point(on_street_gdf, to_street_gdf)
    
    # Filter the segments based on the orientation of the line formed by the intersections
    filtered_segments_gdf = filter_segments_between_points(on_street_gdf, on_from_point, on_to_point)
    
    # dissolve so the entire corridor is one feature
    dissolved_gdf = filtered_segments_gdf.dissolve()
    
    # add on/from/to street data to the feature
    dissolved_gdf['On'] = on_street
    dissolved_gdf['From'] = from_street
    dissolved_gdf['To'] = to_street

    return dissolved_gdf



In [228]:
## Run code to test ##

on_street = ('Halsted St')
from_street = ('98th St')
to_street = ('74th St')

test_gdf = extract_street_segments(gdf, on_street, from_street, to_street)

test_gdf.explore()

## Map a list of on/from/to streets

In [229]:
# read the original on/from/to list as a DataFrame
oft_df = pd.read_csv(on_from_to_list_path)

# Set up a dataframe to list items that cannot be mapped
oft_unmapped = pd.DataFrame()


In [230]:
def get_oft_geometry(row, gdf):
    '''Extract street segments based on the on, from, and to streets. 
    Return the extracted geometry data only'''

    try:
        gdf_geometry = extract_street_segments(gdf, row['On'], row['From'], row['To'])
        return gdf_geometry['geometry']

    # if the location can't be mapped, return None.
    except:
        return None



In [234]:

# Add geometry data to the on/from/to list.
oft_df['geometry'] = oft_df.apply(get_oft_geometry, args=(gdf,), axis=1)

# Turn on/from/to list into a GeoDataFrame
oft_gdf = gpd.GeoDataFrame(oft_df, crs="EPSG:3435", geometry='geometry')

# filter out the unmapped locations
unmapped_filter = oft_gdf['geometry'].isna()

# Create a geoDataFrame with only mapped locations
on_from_to_unmapped_gdf = oft_gdf[unmapped_filter]

# create a GeoDataFrame with only unmapped locations
on_from_to_mapped_gdf = oft_gdf[-unmapped_filter]

on_from_to_mapped_gdf.explore()



/opt/homebrew/Caskroom/miniforge/base/envs/cta-bus-tracker-env/lib/python3.11/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/opt/homebrew/Caskroom/miniforge/base/envs/cta-bus-tracker-env/lib/python3.11/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/opt/homebrew/Caskroom/miniforge/base/envs/cta-bus-tracker-env/lib/python3.11/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/opt/homebrew/Caskroom/miniforge/base/envs/cta-bus-tracker-env/lib/python3.11/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/opt/homebrew/Caskroom/miniforge/base/envs/cta-bus-tracker-env/lib/python3.11/site-packages/shapely/set_oper

In [232]:
on_from_to_unmapped_gdf

,ID,Orig_ON,Orig_FROM,Orig_TO,On,From,To,geometry
2,3,S. Normal Ave. (500W),W. 59th St. (5900S),W. 63rd St. (6300S),NORMAL AVE,59TH ST,63RD ST,None
3,4,. Fort Dearborn Dr. (),. Dead end (),E. 31st St. (3100S),FORT DEARBORN,CERMAK RD,31ST ST,None
41,42,E. 82nd St. (8200S),S. Yates Ave. (2435E),S. Marquette Ave. (2700E),82ND ST,YATES AVE,MARQUETTE AVE,None
46,47,E. 107th St. (10700S),S. Mackinaw Ave. (3332E),S. Dead End (3910E),107TH ST,MACKINAW AVE,AVENUE E,None
47,48,W. Henderson St. (3334N),N. Kimball Ave. (3400W),N. Avondale Ave. (1800W),HENDERSON ST,KIMBALL AVE,AVONDALE AVE,None
49,50,E. 136th St. (13600S),S. Avenue O (3432E),S. Avenue K (3624E),136TH ST,AVENUE O,AVENUE K,None
52,53,E. 111th St. (11100S),S. Green Bay Ave. (3400E),s. Avenue G (3800E),111TH ST,GREEN BAY AVE,AVENUE G,None
53,54,E. 133rd St. (13300S),S. Brainard Ave. (2900E),S. Dead End (4032E),133RD ST,BRAINARD AVE,STATE LINE RD,None
63,64,S. Campbell Ave. (2500W),W. Congress Dr. (500S),W. Roosevelt Dr. (1200S),CAMPBELL AVE,CONGRESS DR,ROOSEVELT DR,None
72,73,W. Fulton St. (300N),N. Austin Ave. (6000W),N. Cicero Ave. (4800W),FULTON ST,AUSTIN AVE,CICERO AVE,None


## Export mapped and unmapped streets as geopackages and csv files

In [233]:
# export mapped locations to gpkg
on_from_to_mapped_gdf.to_file(file_path_mapped_gpkg, driver='GPKG', mode='w')
# export mapped locations to csv without geometry column
on_from_to_mapped_gdf.drop(columns=['geometry']).to_csv(file_path_mapped_csv)

# export unmapped locations to gpkg
on_from_to_unmapped_gdf.to_file(file_path_unmapped_gpkg, driver='GPKG', mode='w')
# export unmapped locations to csv without geometry column
on_from_to_unmapped_gdf.drop(columns=['geometry']).to_csv(file_path_unmapped_csv)

